Notebook recording the various options for the relationship between soil water and groundwater explored during model development. These different versions were motivated by issues, using version 1, with simulating in-stream flows during prolonged periosd of the soil water being below field capacity.

In [1]:
# Import modules
%matplotlib inline
import matplotlib.pyplot as plt, mpld3, seaborn as sn, os, matplotlib as mpl
import numpy as np, pandas as pd
from scipy.integrate import odeint
from matplotlib.ticker import MaxNLocator # for sorting out ticks
mpl.rcParams['pdf.fonttype'] = 42

In [2]:
def hydrol_inputs(D_snow_0, f_DDSM, met_df):
    """Function to calculate snow accumulation and melt.
    Calculates total hydrological input to soil box as precipitation as rain + snowmelt
    
    D_snow_0: Initial snow depth (mm)
    f_DDSM: Degree-day factor for snow melt (mm/degree-day deg C)
    met_df: Dataframe of met data with cols T_air, PET, Pptn
    
    Returns met_df with additional columns [P_snow, P_rain, P_melt, D_snow_start
    D_snow_end, P]
    
    Of these, P is the hydrological input to the soil store (mm/d)
    """
    
    # Precipitation falling as snow (mm/d, as water equivalents)
    met_df.loc[:,'P_snow'] = met_df['Pptn'].ix[met_df['T_air']<0]  # = total pptn if air T<0
    met_df['P_snow'].fillna(0, inplace=True)  # otherwise, =0
    
    # Precipitation falling as rain (mm/d)
    met_df['P_rain'] = met_df['Pptn'] - met_df['P_snow']

    # Potential daily snow melt (unlimited by snow pack depth) (mm/day)
    met_df['P_melt'] = f_DDSM*(met_df['T_air']-0)
    met_df['P_melt'][met_df['P_melt']<0]=0  # Set negative values to 0 (i.e. only melt when T_air>0)

    # Snow pack depth (mm), as end of day depth = start of day depth + inputs - melt, where melt is
    # limited by the depth wherever necessary.
    met_df['D_snow_start'], met_df['D_snow_end'] = np.nan, np.nan # Set-up
    # First time-step manually, to take initial condition into account
    met_df.ix[0,'D_snow_start'] = D_snow_0 # Assign user-supplied starting depth to first row
    met_df.ix[0,'P_melt'] = np.minimum(met_df.ix[0,'P_melt'],met_df.ix[0,'D_snow_start']) # Melt limited by depth
    met_df.ix[0,'D_snow_end'] = (met_df.ix[0,'D_snow_start']+
                                met_df.ix[0,'P_snow']-met_df.ix[0,'P_melt']) # Change over day
    # Calculate for subsequent days
    for idx in range (1,len(met_df)):
        met_df.ix[idx,'D_snow_start'] = met_df.ix[idx-1,'D_snow_end']
        met_df.ix[idx,'P_melt'] = np.minimum(met_df.ix[idx,'P_melt'],met_df.ix[idx,'D_snow_start'])
        met_df.ix[idx,'D_snow_end'] = met_df.ix[idx,'D_snow_start']+met_df.ix[idx,'P_snow']-met_df.ix[idx,'P_melt']

    # Hydrological input to soil box
    met_df.loc[:,'P'] = met_df['P_rain'] + met_df['P_melt']
    
    return met_df

In [3]:
# VERSION 1: two linear reservoirs (soil and groudnwater). Soil water flow only
# when soil water level is above field capacity. No minimum groundwater flow

# Result: Pretty good, but not good at simultaing the recession or low flows.
# Groundwater 'drying out' too quickly.

def hydro_model_1(met_df, ics, p, period, step_len=1):
    """ The hydrological model
            met_df         Dataframe containing columns 'Rainfall_mm' and 'PET_mm',
                           with datetime index
            ics            Vector of initial conditions [Vs0, Vg0]
            p              Series of parameter values (index = param name)
            period         Vector of [start, end] dates [yyyy-mm-dd, yyyy-mm-dd]
            step_len       Length of each step in the input dataset (days)

        Returns a dataframe with column headings
        [Vs, Qs, Vg, Qg, Ds, Dg, Sim_Runoff, Obs_Runoff]
    """
    
    # Function defining the ODE system, to be solved by SCIPY.INTEGRATE.ODEINT
    # at each time step
    def f(y, t, ode_params):
        """ Define ODE system.
                y is list [Vs, Qs, Vg, Qg, Ds, Dg]
                t is an array of time points of interest
                params is a tuple of input values & model params
                (P, E, f_IExcess, alpha, beta, T_s, T_g, fc)
        """
        # Unpack incremental values for Qs and Qg
        Vs_i = y[0]
        Qs_i = y[1]
        Qg_i = y[2]

        # Unpack params
        P, E, f_IExcess, alpha, beta, T_s, T_g, fc = ode_params

        # Soil equations
        dQs_dVs = (((Vs_i - fc)*np.exp(fc - Vs_i))/(T_s*((np.exp(fc-Vs_i) + 1)**2))) + (1/(T_s*(np.exp(fc-Vs_i) + 1)))
        dVs_dt = P*(1-f_IExcess) - alpha*E*(1 - np.exp(-0.02*Vs_i)) - Qs_i
        dQs_dt = dQs_dVs*dVs_dt

        # Groundwater equations
        dQg_dt = (beta*Qs_i - Qg_i)/T_g

        # Total drainage volumes over the timestep
        dDs_dt = (1 - beta)*Qs_i
        dDg_dt = Qg_i

        # Add results of equations to an array
        res = np.array([dVs_dt, dQs_dt, dQg_dt, dDs_dt, dDg_dt])

        return res
    # -------------------------------------------------------------------------

    # Unpack initial conditions
    Vs0, Qg0, Qr0 = ics
    Qs0 = (Vs0 - p['fc'])/(p['T_s']*(1 + np.exp(p['fc'] - Vs0)))
    
    # Time points to evaluate ODEs at. We're only interested in the start and the end of each step
    ti = [0, step_len]

    # Lists to store output
    output_ODEs = []
    output_rest = []

    # Loop over met data
    for idx in range(len(met_df)):

        # Get P and E for this day
        P = met_df.ix[idx, 'P']
        E = met_df.ix[idx, 'PET']

        # Calculate infiltration excess and add to results
        Qq = p['f_IExcess']*P
        output_rest.append(Qq)

        # Vector of initial conditions (Ds and Dg always 0 at start of time step)
        y0 = [Vs0, Qs0, Qg0, 0., 0.]

        # Model parameters plus rainfall and ET, for input to solver
        ode_params = np.array([P, E, p['f_IExcess'], p['alpha'], p['beta'], p['T_s'],p['T_g'], p['fc']])

        # Solve
        y = odeint(f, y0, ti, args=(ode_params,), rtol=0.01, mxstep=5000)

        # Extract values for end of step
        res = y[1]

        # Numerical errors may result in very tiny values <0
        # set these back to 0
        res[res<0] = 0
        output_ODEs.append(res)

        # Update initial conditions for next step
        Vs0 = res[0]
        Qs0 = res[1]
        Qg0 = res[2]

    # Build a dataframe of ODE results
    df1 = pd.DataFrame(data=np.vstack(output_ODEs),
                      columns=['Vs', 'Qs', 'Qg', 'Ds', 'Dg'],
                      index=met_df.index)

    # Dataframe of non ODE results
    df2 = pd.DataFrame(data=np.vstack(output_rest), columns=['Qq'],
                     index=met_df.index)

    # Concatenate results dataframes
    df = pd.concat([df1,df2], axis=1)

    # Estimate runoff as Ds + Dg
    df['Sim_Runoff_mm'] = df['Ds'] + df['Dg'] + df['Qq']
    df['Sim_Runoff_mm_noIE'] = df['Ds'] + df['Dg']
    
    return df

In [4]:
# VERSION 2: Same as version 1, but with a non-linear groundwater reservoir
# (Vg = T*Qg^b, as described by e.g. Wittenberg '99)

# Result: if set the exponent > 1, then the groundwater is better simulated,
# but still not good. Need it to better linked to soil water level (even when
# soil water is less than field capacity). Also, the values of b that help the
# simulation look more realistic are nowhere near what the literature says they
# should be (<1, around 0.5). Though makes me wonder if I'm getting confused 
# about the constants, therefore look into this more if decide to adopt this.

def hydro_model_2(met_df, ics, p, period, step_len=1):
    """ The hydrological model
            met_df         Dataframe containing columns 'Rainfall_mm' and 'PET_mm',
                           with datetime index
            ics            Vector of initial conditions [Vs0, Vg0]
            p              Series of parameter values (index = param name)
                           Has additional parameter k_g (exponent in v=aQ^k_g)
            period         Vector of [start, end] dates [yyyy-mm-dd, yyyy-mm-dd]
            step_len       Length of each step in the input dataset (days)

        Returns a dataframe with column headings
        [Vs, Qs, Vg, Qg, Ds, Dg, Sim_Runoff, Obs_Runoff]
    """
    
    # Function defining the ODE system, to be solved by SCIPY.INTEGRATE.ODEINT
    # at each time step
    def f(y, t, ode_params):
        """ Define ODE system.
                y is list [Vs, Qs, Vg, Qg, Ds, Dg]
                t is an array of time points of interest
                params is a tuple of input values & model params
                (P, E, f_IExcess, alpha, beta, T_s, T_g, fc, k_g)
        """
        # Unpack incremental values for Qs and Qg
        Vs_i = y[0]
        Qs_i = y[1]
        Vg_i = y[2]
        Qg_i = y[3]

        # Unpack params
        P, E, f_IExcess, alpha, beta, T_s, T_g, fc, k_g = ode_params

        # Soil equations
        dQs_dVs = (((Vs_i - fc)*np.exp(fc - Vs_i))/(T_s*((np.exp(fc-Vs_i) + 1)**2))) + (1/(T_s*(np.exp(fc-Vs_i) + 1)))
        dVs_dt = P*(1-f_IExcess) - alpha*E*(1 - np.exp(-0.02*Vs_i)) - Qs_i
        dQs_dt = dQs_dVs*dVs_dt

        # Groundwater equations
        dQg_dVg = (Vg_i**((1-k_g)/k_g))/(k_g*(T_g**(1/k_g)))  # If k_g = 1, simplifies to 1/T (as expected)
        dVg_dt = beta*Qs_i - (Vg_i/T_g)**1/k_g
        dQg_dt = dQg_dVg*dVg_dt

        # Total drainage volumes over the timestep
        dDs_dt = (1 - beta)*Qs_i
        dDg_dt = Qg_i

        # Add results of equations to an array
        res = np.array([dVs_dt, dQs_dt, dVg_dt, dQg_dt, dDs_dt, dDg_dt])

        return res
    # -------------------------------------------------------------------------

    # Unpack initial conditions
    Vs0, Qg0, Qr0 = ics
    Vg0 = p['T_g']*Qg0**p['k_g']
    Qs0 = (Vs0 - p['fc'])/(p['T_s']*(1 + np.exp(p['fc'] - Vs0)))
    
    # Time points to evaluate ODEs at. We're only interested in the start and the end of each step
    ti = [0, step_len]

    # Lists to store output
    output_ODEs = []
    output_rest = []

    # Loop over met data
    for idx in range(len(met_df)):

        # Get P and E for this day
        P = met_df.ix[idx, 'P']
        E = met_df.ix[idx, 'PET']

        # Calculate infiltration excess and add to results
        Qq = p['f_IExcess']*P
        output_rest.append(Qq)

        # Vector of initial conditions (Ds and Dg always 0 at start of time step)
        y0 = [Vs0, Qs0, Vg0, Qg0, 0., 0.]

        # Model parameters plus rainfall and ET, for input to solver
        ode_params = np.array([P, E, p['f_IExcess'], p['alpha'], p['beta'], p['T_s'],
                               p['T_g'], p['fc'], p['k_g']])

        # Solve
        y = odeint(f, y0, ti, args=(ode_params,), rtol=0.01, mxstep=5000)

        # Extract values for end of step
        res = y[1]

        # Numerical errors may result in very tiny values <0
        # set these back to 0
        res[res<0] = 0
        output_ODEs.append(res)

        # Update initial conditions for next step
        Vs0 = res[0]
        Qs0 = res[1]
        Vg0 = res[2]
        Qg0 = res[3]

    # Build a dataframe of ODE results
    df1 = pd.DataFrame(data=np.vstack(output_ODEs),
                      columns=['Vs', 'Qs', 'Vg', 'Qg', 'Ds', 'Dg'],
                      index=met_df.index)

    # Dataframe of non ODE results
    df2 = pd.DataFrame(data=np.vstack(output_rest), columns=['Qq'],
                     index=met_df.index)

    # Concatenate results dataframes
    df = pd.concat([df1,df2], axis=1)

    # Estimate runoff as Ds + Dg
    df['Sim_Runoff_mm'] = df['Ds'] + df['Dg'] + df['Qq']
    df['Sim_Runoff_mm_noIE'] = df['Ds'] + df['Dg']
    
    return df

VERSION 3: Allow runoff from the soil box when the soil water level is below
field capacity, using the same function to limit its value as is used for the
AET calculation. Allows runoff to the stream and percolation to groundwater
when the soil water is below field capacity, but at a reduced rate.

Result: Too much lost from the soil water, and therefore groundwater and soil
water flows are too high, resulting in discharge which is also too high. The
only way to make discharge around the right level is to reduce field capacity
to values which essentially stop it physically representing field capacity
any more (e.g. 10mm), but even then the fit isn't very good.

Conclude: Need soil water flow to be very limited below field capacity.
This function isn't suitable. Need a sigmoid. Definitely an area for future exploration.
I did some exploring of possible sigmoids, but you run into difficulties with generalising
the shape parameter to define them, given that fc can vary. A possible future solution
could be to have TWO sigmoids, with the second being very steep and kicking in once the
soil water goes below the permanent wilting point. This would likely introduce one
additional user-specified parameter to define the shape of the main sigmoid controlling
soil water flow between the wilting point and field capacity, though may be able to fix this...
Highlight as something to be investigated in the future!

N.B. I've been playing around with the k shape parameter in this function; it's
currently calculated assuming Q is 0.5 of potential when V is at FC.

In [5]:
def hydro_model_3(met_df, ics, p, period, step_len=1):
    """ The hydrological model

            met_df         Dataframe containing columns 'Rainfall_mm' and 'PET_mm', with datetime index
            ics            Vector of initial conditions [Vs0, Vg0]
            p              Series of parameter values (index = param name)
            period         Vector of [start, end] dates [yyyy-mm-dd, yyyy-mm-dd]
            step_len       Length of each step in the input dataset (days)

        Returns a dataframe with column headings
        [Vs, Qs, Qg, Ds, Dg, Sim_Runoff, Obs_Runoff]
    """
    # ------------------------------------------------------------------------
    # Define the ODE system
    def f(y, t, ode_params):
        """ Define ODE system.
                y is list [Vs, Qs, Qg, Ds, Dg]
                t is an array of time points of interest
                params is a tuple of input values & model params
                (P, E, f_IExcess, alpha, beta, T_s, T_g, fc)
        """
        # Unpack incremental values for Qs and Qg
        Vs_i = y[0]
        Qs_i = y[1]
        Qg_i = y[2]
        
        # Unpack params
        P, E, f_IExcess, alpha, beta, T_s, T_g, fc, k = ode_params
           
        # Soil water equations
        dVs_dt = P*(1-f_IExcess) - alpha*E*(1-np.exp(-k*Vs_i)) - (Vs_i/T_s)*(1-np.exp(-k*Vs_i))       
        dQs_dV = np.exp(-k*(Vs_i**2))*(2*k*(Vs_i**2)+np.exp(k*(Vs_i**2))-1)*(1/T_s)
        dQs_dt = dQs_dV*dVs_dt
        
        # Groundwater equations
        dQg_dt = (beta*Qs_i - Qg_i)/T_g
        
        # Total drainage volumes over the timestep
        dDs_dt = (1 - beta)*Qs_i
        dDg_dt = Qg_i
        
        # Add results of equations to an array
        res = np.array([dVs_dt, dQs_dt, dQg_dt, dDs_dt, dDg_dt])
        
        return res
    # -------------------------------------------------------------------------
    # Calculate the value of the exponential shape parameter, k
    k = np.log(0.5)/-p['fc']
	
    # Unpack initial conditions
    Vs0, Qg0, Qr0 = ics
    Qs0 = Vs0*(1/p['T_s'])*(1-np.exp(-k*Vs0))
	
    # Time points to evaluate ODEs at. We're only interested in the start and
    # the end of each step
    ti = [0, step_len]

    # Lists to store output
    output_ODEs = []
    output_rest = []

    # Loop over met data
    for idx in range(len(met_df)):

        # Get P and E for this day
        P = met_df.ix[idx, 'P']
        E = met_df.ix[idx, 'PET']

        # Calculate infiltration excess and add to results
        Qq = p['f_IExcess']*P
        output_rest.append(Qq)

        # Vector of initial conditions (Ds and Dg always 0 at start of time step)
        y0 = [Vs0, Qs0, Qg0, 0., 0.]

        # Model parameters plus rainfall and ET, for input to solver
        ode_params = np.array([P, E, p['f_IExcess'], p['alpha'], p['beta'], p['T_s'],
                               p['T_g'], p['fc'], k])

        # Solve
        y = odeint(f, y0, ti, args=(ode_params,), rtol=0.01, mxstep=5000)

        # Extract values for end of step
        res = y[1]

        # Numerical errors may result in very tiny values <0
        # set these back to 0
        res[res<0] = 0
        output_ODEs.append(res)

        # Update initial conditions for next step
        Vs0 = res[0]
        Qs0 = res[1]
        Qg0 = res[2]

    # Build a dataframe of ODE results
    df1 = pd.DataFrame(data=np.vstack(output_ODEs),
                      columns=['Vs', 'Qs', 'Qg', 'Ds', 'Dg'],
                      index=met_df.index)

    # Dataframe of non ODE results
    df2 = pd.DataFrame(data=np.vstack(output_rest), columns=['Qq'],
                     index=met_df.index)

    # Concatenate results dataframes
    df = pd.concat([df1,df2], axis=1)

    # Estimate runoff as Ds + Dg
    df['Sim_Runoff_mm'] = df['Ds'] + df['Dg'] + df['Qq']
    df['Sim_Runoff_mm_noIE'] = df['Ds'] + df['Dg']

    return df

In [6]:
# VERSION 4

# Same as version 1, but includes a minimum groundwater flow threshold

def hydro_model_4(met_df, ics, p, period, step_len=1):
    """ The hydrological model
            met_df         Dataframe containing columns 'Rainfall_mm' and 'PET_mm',
                           with datetime index
            ics            Vector of initial conditions [Vs0, Vg0]
            p              Series of parameter values (index = param name)
            period         Vector of [start, end] dates [yyyy-mm-dd, yyyy-mm-dd]
            step_len       Length of each step in the input dataset (days)

        Returns a dataframe with column headings
        [Vs, Qs, Vg, Qg, Ds, Dg, Sim_Runoff, Obs_Runoff]
    """
    
    # Function defining the ODE system, to be solved by SCIPY.INTEGRATE.ODEINT
    # at each time step
    def f(y, t, ode_params):
        """ Define ODE system.
                y is list [Vs, Qs, Vg, Qg, Ds, Dg]
                t is an array of time points of interest
                params is a tuple of input values & model params
                (P, E, f_IExcess, alpha, beta, T_s, T_g, fc)
        """
        # Unpack incremental values for Qs and Qg
        Vs_i = y[0]
        Qs_i = y[1]
        Qg_i = y[2]

        # Unpack params
        P, E, f_IExcess, alpha, beta, T_s, T_g, fc = ode_params

        # Soil equations
        dQs_dVs = (((Vs_i - fc)*np.exp(fc - Vs_i))/(T_s*((np.exp(fc-Vs_i) + 1)**2))) + (1/(T_s*(np.exp(fc-Vs_i) + 1)))
        dVs_dt = P*(1-f_IExcess) - alpha*E*(1 - np.exp(-0.02*Vs_i)) - Qs_i
        dQs_dt = dQs_dVs*dVs_dt

        # Groundwater equations
        dQg_dt = (beta*Qs_i - Qg_i)/T_g

        # Total drainage volumes over the timestep
        dDs_dt = (1 - beta)*Qs_i
        dDg_dt = Qg_i

        # Add results of equations to an array
        res = np.array([dVs_dt, dQs_dt, dQg_dt, dDs_dt, dDg_dt])

        return res
    # -------------------------------------------------------------------------

    # Unpack initial conditions
    Vs0, Qg0, Qr0 = ics
    Qs0 = (Vs0 - p['fc'])/(p['T_s']*(1 + np.exp(p['fc'] - Vs0)))
    
    # Time points to evaluate ODEs at. We're only interested in the start and the end of each step
    ti = [0, step_len]

    # Lists to store output
    output_ODEs = []
    output_rest = []

    # Loop over met data
    for idx in range(len(met_df)):

        # Get P and E for this day
        P = met_df.ix[idx, 'P']
        E = met_df.ix[idx, 'PET']

        # Calculate infiltration excess and add to results
        Qq = p['f_IExcess']*P
        output_rest.append(Qq)

        # Vector of initial conditions (Ds and Dg always 0 at start of time step)
        y0 = [Vs0, Qs0, Qg0, 0., 0.]

        # Model parameters plus rainfall and ET, for input to solver
        ode_params = np.array([P, E, p['f_IExcess'], p['alpha'], p['beta'], p['T_s'],p['T_g'], p['fc']])

        # Solve
        y = odeint(f, y0, ti, args=(ode_params,), rtol=0.01, mxstep=5000)

        # Extract values for end of step
        res = y[1]

        # Numerical errors may result in very tiny values <0
        # set these back to 0
        res[res<0] = 0
        output_ODEs.append(res)

        # Update initial conditions for next step
        Vs0 = res[0]
        Qs0 = res[1]
        Qg0 = res[2]
        # Re-set groundwater to user-supplied minimum flow at start of each time step. Non-ideal
        # solution to the problem or maintaining stream flow during baseflow conditions.
        if p['Qg_min'] > res[2]:
            Qg0 = p['Qg_min']
        else:
            Qg0 = res[2]

    # Build a dataframe of ODE results
    df1 = pd.DataFrame(data=np.vstack(output_ODEs),
                      columns=['Vs', 'Qs', 'Qg', 'Ds', 'Dg'],
                      index=met_df.index)

    # Dataframe of non ODE results
    df2 = pd.DataFrame(data=np.vstack(output_rest), columns=['Qq'],
                     index=met_df.index)

    # Concatenate results dataframes
    df = pd.concat([df1,df2], axis=1)

    # Estimate runoff as Ds + Dg
    df['Sim_Runoff_mm'] = df['Ds'] + df['Dg'] + df['Qq']
    df['Sim_Runoff_mm_noIE'] = df['Ds'] + df['Dg']
    
    return df

In [26]:
# Model parameters
# Units: L_reach in mm for now. a_Q and b_Q are for m/s vs m3/s
p1 = pd.Series({'A_catch': 51.7, 'L_reach':10000. ,'fc':290.,
    'alpha':0.80, 'beta':0.6,'f_IExcess':0.015, 'T_s':3.,'T_g':65., 'Qg_min':0, 'k_g':1.2,
    'a_Q':0.5, 'b_Q':0.5,'Qg0_init':1.0, 'Qr0_init': 1.0})

p2 = pd.Series({'A_catch': 51.7, 'L_reach':10000. ,'fc':290.,
    'alpha':0.80, 'beta':0.6,'f_IExcess':0.015, 'T_s':3.,'T_g':65., 'Qg_min':0, 'k_g':1.2,
    'a_Q':0.5, 'b_Q':0.5,'Qg0_init':1.0, 'Qr0_init': 1.0})

# Simulation period
st_dt = '2004-01-01'  # Start date
end_dt = '2005-12-31' # End date

In [9]:
# READ IN DATA
# Desktop
metdata_fpath = r'M:\Working\NewModel\ModelInputs\Tar_AvMetData_1981-2010.csv'
Qobsdata_fpath = r'M:\Working\NewModel\ModelInputs\obs_csvs\Coull_9amDailyMeanQ_oldRating.xlsx'
# Met
met_df = pd.read_csv(metdata_fpath, parse_dates=True, dayfirst=True, index_col=0)
met_df = met_df.truncate(before=st_dt, after=end_dt)  # Truncate to the desired period
met_df = hydrol_inputs(0.0, 2.74, met_df)
# Obs
Qobs_df = pd.read_excel(Qobsdata_fpath, sheetname=str(1), index_col=0, parse_dates=True, dayfirst=True)
Qobs_df = Qobs_df.truncate(before=st_dt, after=end_dt)
Qobs_df['Obs_Runoff_mm'] = Qobs_df['Q']*86400/(1000*p2.A_catch)

In [27]:
df1 = hydro_model_1(met_df, p1[['fc','Qg0_init','Qr0_init']], p1, period=[st_dt, end_dt], step_len=1)
df2 = hydro_model_2(met_df, p2[['fc','Qg0_init','Qr0_init']], p2, period=[st_dt, end_dt], step_len=1)

In [29]:
folder_path = r'M:\Working\NewModel\ModelOutputs\Figs\Hydrol_Development'
sn.set_context('paper') # 'notebook' for on-line viewing; 'paper' for smaller fonts
# Plot results
fig = plt.figure(figsize=(7.5,2))
# fig = plt.figure(figsize=(15,5))
ax = fig.add_subplot(1,1,1)
Qobs_df['Obs_Runoff_mm'].plot(ax=ax, color='0.5', label='Observed', lw=1.5) # Plot observed

# # Fig 1: Effect of IE
# df1['Sim_Runoff_mm'].plot(ax=ax, color='r', label='Simulated, with quick flow', lw=1)
# df2['Sim_Runoff_mm_noIE'].plot(ax=ax, color='b', label='Simulated, no quick flow', lw=1)

# Fig 2: Linear vs non-linear GW. Run models 1 and 2; Qgmin=0 for both
df1['Sim_Runoff_mm'].plot(ax=ax, color='r', label='Simulated, linear GW', lw=1, logy=True)
df2['Sim_Runoff_mm'].plot(ax=ax, color='b', label='Simulated, non-linear GW', lw=1)
outGraph = os.path.join(folder_path,"Fig_H2_LinearGW.png")

# Fig 3: Threshold. Run models df1=4, Qgmin=0; df2=4, Qgmin=0.4 mm/d
# df1['Sim_Runoff_mm'].plot(ax=ax, color='r', label='Sim, no GW threshold', lw=1, logy=True)
# df2['Sim_Runoff_mm'].plot(ax=ax, color='b', label='Sim, inc. GW threshold', lw=1)
# outGraph = os.path.join(folder_path,"Fig_H3_GW_threshold.png")

# Tidy up plot
ax.legend(loc='best')
# ax.yaxis.set_major_locator(MaxNLocator(nbins=5, prune='upper'))
plt.ylabel('Runoff (mm day$^{-1}$)')  # y-axis label
plt.xlabel("")

# plt.show()
plt.savefig(outGraph, bbox_inches='tight', dpi=300)
plt.close()

In [ ]:
df1.head()

In [ ]:
df2.head()